<a href="https://colab.research.google.com/github/Xu-Ruixin/Multilingual-Phoneme-Recognition/blob/main/CS2FI_Phoneme_Recognition_(Piloting).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Dec  3 18:01:19 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8    12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%%capture
!pip install -U datasets
!pip install -U transformers[torch]
!pip install torchaudio
!pip install librosa
!pip install -U huggingface_hub
!pip install jiwer
!pip install accelerate -U
!pip install phonemizer
!apt install git-lfs
!sudo apt-get install festival espeak-ng mbrola
!apt search mbrola
!apt install git-lfs

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from datasets import load_dataset, load_metric, Audio

# common_voice_train = load_dataset("facebook/voxpopuli", "fi", split="train+validation")
common_voice_train = load_dataset("facebook/voxpopuli", "cs", split="validation")
common_voice_test = load_dataset("facebook/voxpopuli", "fi", split="test")
common_voice_test_2 = common_voice_test

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
common_voice_train

Dataset({
    features: ['audio_id', 'language', 'audio', 'raw_text', 'normalized_text', 'gender', 'speaker_id', 'is_gold_transcript', 'accent'],
    num_rows: 1103
})

In [ ]:
sentence = "normalized_text"
col_names = list(common_voice_train.features.keys())
col_names.remove(sentence)
col_names.remove("audio")
common_voice_train = common_voice_train.remove_columns(col_names)
common_voice_test = common_voice_test.remove_columns(col_names)

In [ ]:
common_voice_train

Dataset({
    features: ['audio', 'normalized_text'],
    num_rows: 1103
})

In [ ]:
common_voice_test

Dataset({
    features: ['audio', 'normalized_text'],
    num_rows: 478
})

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(common_voice_train, num_examples=1)

,audio,normalized_text
0,"{'path': '/root/.cache/huggingface/datasets/downloads/extracted/878bb93f50316d028ef5fbc7cc348b2f97af8adc6c081131daea7c82daa38113/dev_part_0/20171116-0900-PLENARY-5-cs_20171116-10:47:55_9.wav', 'array': [-0.00103759765625, -0.003753662109375, -0.002716064453125, 0.00360107421875, -0.001251220703125, -0.004119873046875, -0.000823974609375, 0.003143310546875, 0.001434326171875, -0.00201416015625, -0.0062255859375, -0.00384521484375, 0.002044677734375, 6.103515625e-05, -0.000244140625, 0.002288818359375, 0.003997802734375, 0.001495361328125, 0.0003662109375, -0.0040283203125, -0.00323486328125, -0.00067138671875, 0.002349853515625, 0.006011962890625, 0.001861572265625, -0.003448486328125, -0.00250244140625, 0.00518798828125, 0.000885009765625, -0.002410888671875, -0.00067138671875, -0.00042724609375, 0.000274658203125, 0.001495361328125, -0.00146484375, -0.003173828125, 0.00274658203125, 0.003326416015625, -0.002166748046875, -0.002838134765625, 0.001861572265625, 6.103515625e-05, 0.001434326171875, 0.0009765625, -0.001739501953125, -0.001007080078125, 0.0008544921875, 0.002716064453125, 0.000701904296875, -0.00164794921875, -0.001068115234375, 0.003265380859375, 0.000152587890625, -0.001220703125, 0.0020751953125, -0.00030517578125, -0.000823974609375, 0.00048828125, -0.000518798828125, -0.00067138671875, -0.001953125, -0.002288818359375, 0.001800537109375, -0.001556396484375, -0.0025634765625, 0.00225830078125, 0.00067138671875, -0.00146484375, -0.0008544921875, 0.001129150390625, 0.000335693359375, -0.00048828125, -0.0006103515625, 0.002288818359375, 0.00225830078125, 0.001312255859375, 0.002593994140625, -0.001220703125, -0.00384521484375, -0.003082275390625, -0.000396728515625, 0.00079345703125, 0.003509521484375, 0.000732421875, -0.0020751953125, -0.000885009765625, -0.0008544921875, 0.000518798828125, 0.0040283203125, 0.00390625, -0.003021240234375, 0.000244140625, 0.001800537109375, -0.001861572265625, 0.002044677734375, 0.00213623046875, -0.0008544921875, 0.00177001953125, -0.00048828125, -0.002777099609375, ...], 'sampling_rate': 16000}",měla by zároveň mobilizovat celé mezinárodní společenství především sousední a další státy regionu aby každý podle svých možností přispěl ke konci somálska na území které kontrolují více teroristé než vláda somálska.


In [ ]:
import re
chars_to_remove_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\']'

def remove_special_characters(batch):
    batch[sentence] = re.sub(chars_to_remove_regex, '', batch[sentence]).lower()
    return batch

In [ ]:
common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)

Map:   0%|          | 0/1103 [00:00<?, ? examples/s]

Map:   0%|          | 0/478 [00:00<?, ? examples/s]

In [ ]:
show_random_elements(common_voice_train, 1)

,audio,normalized_text
0,"{'path': '/root/.cache/huggingface/datasets/downloads/extracted/878bb93f50316d028ef5fbc7cc348b2f97af8adc6c081131daea7c82daa38113/dev_part_0/20170405-0900-PLENARY-13-cs_20170405-16:32:47_1.wav', 'array': [-0.0001220703125, -0.00067138671875, -0.00018310546875, 0.000518798828125, 0.0008544921875, 0.000579833984375, 0.000213623046875, 0.0001220703125, -6.103515625e-05, -0.000396728515625, -0.0006103515625, -6.103515625e-05, 0.000518798828125, 0.000518798828125, 0.000396728515625, 9.1552734375e-05, 0.0, 0.000274658203125, 0.00042724609375, 3.0517578125e-05, -0.0003662109375, -6.103515625e-05, 0.0001220703125, -6.103515625e-05, -0.0006103515625, -0.0009765625, -0.000457763671875, 6.103515625e-05, -0.0001220703125, -0.00042724609375, -0.00054931640625, -0.000396728515625, -0.000213623046875, 0.0001220703125, 0.0001220703125, 3.0517578125e-05, 0.0006103515625, 0.000762939453125, 0.0003662109375, 0.00030517578125, 0.00030517578125, 0.000457763671875, 0.0003662109375, 9.1552734375e-05, 0.000152587890625, 0.0003662109375, 0.000244140625, -0.00018310546875, 9.1552734375e-05, 0.000396728515625, -9.1552734375e-05, -0.0003662109375, -0.000274658203125, 0.000152587890625, 0.00054931640625, 0.000244140625, -0.0003662109375, -0.00048828125, -9.1552734375e-05, 0.000213623046875, 0.000640869140625, 0.000335693359375, -0.000518798828125, -0.000244140625, -0.0001220703125, -0.0006103515625, -0.00079345703125, -0.000457763671875, 6.103515625e-05, 6.103515625e-05, -0.000244140625, -0.00042724609375, 0.000244140625, 0.000885009765625, -6.103515625e-05, -0.0009765625, -0.00079345703125, -0.000274658203125, -0.0001220703125, -0.00054931640625, -0.000518798828125, 0.000274658203125, 0.000732421875, 0.0, -0.000885009765625, -0.0006103515625, -6.103515625e-05, -3.0517578125e-05, -0.00048828125, -0.000579833984375, -0.0001220703125, 6.103515625e-05, -0.000152587890625, -0.00079345703125, -0.001312255859375, -0.001129150390625, -0.000762939453125, -0.000274658203125, -0.0001220703125, -0.000244140625, ...], 'sampling_rate': 16000}",tento fenomén je potřeba efektivně konfrontovat a na základě objektivních faktů tyto dezinformace vyvracet jak na evropské tak na národní úrovni


In [ ]:
from transformers import Wav2Vec2Processor

# processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-lv-60-espeak-cv-ft")

preprocessor_config.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/321 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/4.64k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

In [ ]:
common_voice_train = common_voice_train.cast_column("audio", Audio(sampling_rate=16_000))
common_voice_test = common_voice_test.cast_column("audio", Audio(sampling_rate=16_000))

In [ ]:
common_voice_train[0]["audio"]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/878bb93f50316d028ef5fbc7cc348b2f97af8adc6c081131daea7c82daa38113/dev_part_0/20171115-0900-PLENARY-12-cs_20171115-13:38:25_3.wav',
 'array': array([-0.0010376 , -0.00476074, -0.00054932, ..., -0.00021362,
        -0.00018311, -0.0010376 ]),
 'sampling_rate': 16000}

In [ ]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(common_voice_train)-1)

print(common_voice_train[rand_int]["normalized_text"])
ipd.Audio(data=common_voice_train[rand_int]["audio"]["array"], autoplay=True, rate=16000)

a v české republice ale i v jiných státech se k tomu připojuje i problematika suburbanizovaných území to znamená těch které jsou v aglomeračních prstencích a kde doprava je skutečně velmi zásadní pro místní obyvatele a pro její dopady na zdravý životní styl


In [ ]:
rand_int = random.randint(0, len(common_voice_train)-1)

print("Target text:", common_voice_train[rand_int]["normalized_text"])
print("Input array shape:", common_voice_train[rand_int]["audio"]["array"].shape)
print("Sampling rate:", common_voice_train[rand_int]["audio"]["sampling_rate"])

Target text: chtěla bych vyzdvihnout také podporu mobility výzkumných pracovníků v evropě podporu investic do evropských výzkumných infrastruktur budování komplexní sítě schopné řešit tematické výzvy čímž je celkově posílen evropský výzkumný prostor
Input array shape: (222709,)
Sampling rate: 16000


In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    with processor.as_target_processor():
        batch["labels"] = processor(batch["normalized_text"]).input_ids
    return batch

In [ ]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names)

Map:   0%|          | 0/1103 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/478 [00:00<?, ? examples/s]

In [ ]:
max_input_length_in_sec = 5.0
common_voice_train = common_voice_train.filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])

common_voice_test = common_voice_test.filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])

Filter:   0%|          | 0/1103 [00:00<?, ? examples/s]

Filter:   0%|          | 0/478 [00:00<?, ? examples/s]

In [ ]:
common_voice_train

Dataset({
    features: ['input_values', 'input_length', 'labels'],
    num_rows: 185
})

In [ ]:
common_voice_test

Dataset({
    features: ['input_values', 'input_length', 'labels'],
    num_rows: 64
})

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
wer_metric = load_metric("wer")

<ipython-input-26-605de035dfd7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")


In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import Wav2Vec2ForCTC

dropout_ratio = 0.2

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-lv-60-espeak-cv-ft",
    attention_dropout=dropout_ratio,
    hidden_dropout=dropout_ratio,
    feat_proj_dropout=dropout_ratio,
    mask_time_prob=0.05,
    layerdrop=0.0,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
)

config.json:   0%|          | 0.00/1.86k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-lv-60-espeak-cv-ft were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-lv-60-espeak-cv-ft and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably

In [ ]:
model.freeze_feature_extractor()

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1913: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5. Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [ ]:
from transformers import TrainingArguments

repo_name = "cs2fi-wav2vec2-large-xls-r-300m-cs-colab-phoneme"

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=50,
  gradient_checkpointing=True,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=100,
  learning_rate=1e-5,
  warmup_steps=500,
  save_total_limit=2,
  push_to_hub=True,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor.feature_extractor,
)

To avoid inactivity: inspect -> console -> copy-paste

```javascript
function ConnectButton(){
    console.log("Connect pushed");
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click()
}
setInterval(ConnectButton,60000);
```

In [ ]:
trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


{'eval_loss': 4.26776647567749,
 'eval_wer': 0.6596173212487412,
 'eval_runtime': 10.8154,
 'eval_samples_per_second': 5.917,
 'eval_steps_per_second': 0.74}

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
100,3.806100,2.743654,0.569319
200,2.871000,2.420894,0.531386
300,2.525000,2.250783,0.525344


/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Tran

TrainOutput(global_step=300, training_loss=3.067368367513021, metrics={'train_runtime': 1072.4566, 'train_samples_per_second': 8.625, 'train_steps_per_second': 0.28, 'total_flos': 1.214755319938604e+18, 'train_loss': 3.067368367513021, 'epoch': 50.0})

In [ ]:
trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


{'eval_loss': 2.2507834434509277,
 'eval_wer': 0.5253440751930178,
 'eval_runtime': 4.4772,
 'eval_samples_per_second': 14.294,
 'eval_steps_per_second': 1.787,
 'epoch': 50.0}

In [ ]:
input_dict = processor(common_voice_test[0]["input_values"], return_tensors="pt", padding=True)
print(input_dict)

logits = model(input_dict.input_values.to("cuda")).logits
print(logits)

pred_ids = torch.argmax(logits, dim=-1)[0]
print(pred_ids)

print("Prediction:")
print(processor.decode(pred_ids))

print("\nReference:")
print(common_voice_test_2[0][sentence].lower())

It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


{'input_values': tensor([[-0.0235, -0.0708, -0.0633,  ..., -0.3788,  0.4772, -0.3712]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]], dtype=torch.int32)}
tensor([[[ 13.5938, -19.2344, -18.8750,  ..., -17.7656, -19.2500, -18.4688],
         [ 13.6484, -19.4219, -19.0781,  ..., -17.9531, -19.4375, -18.6406],
         [ 13.6875, -19.0938, -18.7031,  ..., -17.6562, -19.1562, -18.3125],
         ...,
         [  8.1953, -12.8672, -13.0859,  ..., -12.2109, -13.2188, -12.3281],
         [  8.3438, -12.8438, -13.3750,  ..., -12.6641, -13.3281, -12.6094],
         [  8.6797, -12.9062, -13.2969,  ..., -12.5547, -13.1719, -12.7188]]],
       device='cuda:0', grad_fn=<ToCopyBackward0>)
tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  5,  0,  0,  0, 46,  0,
         0,  0, 46,  0,  0,  4,  0, 41,  0,  0,  0,  0,  0,  0,  0,  0, 13,  0,
        44,  0,  0,  0,  0,  5,  0,  0,  0,  0, 18, 18,  0, 46,  0,  0,  0,  0,
         0,  0,  0,  6,  0, 14,  0,  0,  0,  8,  0, 10,  0,  0,  0,  

# Push

In [ ]:
# save the checkpoint for inference
trainer.push_to_hub()

events.out.tfevents.1701628976.239a655b1794.525.1:   0%|          | 0.00/406 [00:00<?, ?B/s]

'https://huggingface.co/Roxysun/cs2fi-wav2vec2-large-xls-r-300m-cs-colab-phoneme/tree/main/'

# Test Fine-tuned model

In [ ]:
model = Wav2Vec2ForCTC.from_pretrained("Roxysun/"+repo_name).to("cuda")
# processor = Wav2Vec2Processor.from_pretrained(repo_name)
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-lv-60-espeak-cv-ft")

config.json:   0%|          | 0.00/2.34k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

In [ ]:
input_dict = processor(common_voice_test[0]["input_values"], return_tensors="pt", padding=True)
print(input_dict)

logits = model(input_dict.input_values.to("cuda")).logits
print(logits)

pred_ids = torch.argmax(logits, dim=-1)[0]
print(pred_ids)


print("Prediction:")
print(processor.decode(pred_ids))

print("\nReference:")
print(common_voice_test_2[0][sentence].lower())

It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


{'input_values': tensor([[-0.0235, -0.0708, -0.0633,  ..., -0.3788,  0.4772, -0.3712]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]], dtype=torch.int32)}
tensor([[[ 13.5894, -19.2164, -18.8709,  ..., -17.7583, -19.2404, -18.4641],
         [ 13.6516, -19.4263, -19.0821,  ..., -17.9542, -19.4447, -18.6502],
         [ 13.6950, -19.1020, -18.7140,  ..., -17.6586, -19.1555, -18.3231],
         ...,
         [  8.1956, -12.8695, -13.0856,  ..., -12.2109, -13.2144, -12.3268],
         [  8.3447, -12.8425, -13.3747,  ..., -12.6697, -13.3276, -12.6073],
         [  8.6821, -12.9027, -13.2928,  ..., -12.5520, -13.1637, -12.7152]]],
       device='cuda:0', grad_fn=<ViewBackward0>)
tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  5,  0,  0,  0, 46,  0,
         0,  0, 46,  0,  0,  4,  0, 41,  0,  0,  0,  0,  0,  0,  0,  0, 13,  0,
        44,  0,  0,  0,  0,  5,  0,  0,  0,  0, 18, 18,  0, 46,  0,  0,  0,  0,
         0,  0,  0,  6,  0, 14,  0,  0,  0,  8,  0, 10,  0,  0,  0,  0,